## Lab 2 line following code:
In the block below, there's a play_video function, the way videos seem to work here is that our cv.VideoCapture is sending in a raw stream which we can iterate through by calling .read() on the stream this returns a little boolean fellow that tells us if everything is okay with the stream or if it's ended, and a frame object which seems to be a numpy array. To end the whole thing, we either wait for the ret to be false in which case we break the loop, or we wait for the q key to be pressed, the waitkey function at the end also set's the speed at which the video gets played back since it's the waiting that causes the frame to be shown. To slow it down we could set a longer wait time.

In [2]:
import numpy as np
import cv2 as cv

raw_feed = cv.VideoCapture('raw_video_feed.mp4')

def play_video(cap):
    ## @brief plays a video 
    #  @param cap cv input stream
    ##
    
    while True:
        ret, frame = cap.read()

        if not ret:
            break
    
        cv.imshow('play video', frame)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

play_video(raw_feed)

#### My first thought's for a line follow function:

Similarily to just playing a video, since I want to do operations on each individual frame I do my stuff in a while loop, this time the function can be called on a video path. Here we use the cv.threshold function which is interesting because it has various tags for how you want to actually threshold. then I just do a for loop on a row near the bottom of the image and try to get the left and right edges of the line. This works but the circle blows up whenever we take a corner since the bottom stretches pretty wide.

In [4]:
def linefollow(video_path):
    vid_stream = cv.VideoCapture(video_path)
    while True:
        ret, frame = vid_stream.read()

        if not ret:
            break

        ret,bnw_frame = cv.threshold(frame,127,255,cv.THRESH_BINARY)

        height, width, ret2 = bnw_frame.shape

        not_seen = True

        for i in range(width):
            if (not_seen and np.array_equal(bnw_frame[height-1, i], [0,0,0])):
                black_left = i
                not_seen = False
            elif (not not_seen and np.array_equal(bnw_frame[height-1, i], [255,255,255])):
                black_right = i
                break

        circle_radius = int((black_right-black_left)/2)
        center_coords = (int((black_right+black_left)/2), int(height-circle_radius))

        cv.circle(frame, center_coords, circle_radius, (0,0,255), -1)

        cv.imshow("video", frame)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    vid_stream.release()
    cv.destroyAllWindows()
    

linefollow('raw_video_feed.mp4')

#### A better line follower:
Here we see some more interesting cv functions, firstly to convert a color image to grayscale we use cvt color with the proper tag. Note the difference between BGR and RGB. Now since the frame is just a 2d array, we select the bottom 10 pixels and call moments on it, note that this bottom part is not just thresholded but inv thresholded so the line should be white. moments returns a dictionary so to get elements from it we use strings as the keys. From the moments we can learn stuff about a shape, namely it's area or the 00 moment, and it's center of mass which is just the first order moment divided by the area.

In [5]:
def linefollow2(video_path):
    vid_stream = cv.VideoCapture(video_path)
    while True:
        ret, frame = vid_stream.read()

        if not ret:
            break

        gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        ret,bnw_frame = cv.threshold(gray_frame,100,255,cv.THRESH_BINARY_INV)

        bottom_part = bnw_frame[-20:, :]

        # moments are a fancy opencv thingy
        moments = cv.moments(bottom_part, binaryImage=True)
        if moments["m00"] != 0:
            cx = int(moments["m10"] / moments["m00"])
            cy = frame.shape[0]-20
        cv.circle(frame, (cx,cy), 20, (0,0,255), -1)
        print((cx,cy))

        cv.imshow("video", frame)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    vid_stream.release()
    cv.destroyAllWindows()

linefollow2('raw_video_feed.mp4')

(85, 220)
(99, 220)
(104, 220)
(111, 220)
(116, 220)
(111, 220)
(103, 220)
(86, 220)
(65, 220)
(70, 220)
(76, 220)
(85, 220)
(93, 220)
(99, 220)
(106, 220)
(113, 220)
(113, 220)
(101, 220)
(91, 220)
(76, 220)
(63, 220)
(57, 220)
(66, 220)
(76, 220)
(81, 220)
(87, 220)
(87, 220)
(100, 220)
(102, 220)
(95, 220)
(88, 220)
(83, 220)
(72, 220)
(67, 220)
(83, 220)
(90, 220)
(97, 220)
(103, 220)
(100, 220)
(96, 220)
(88, 220)
(79, 220)
(61, 220)
(67, 220)
(67, 220)
(81, 220)
(97, 220)
(104, 220)
(106, 220)
(99, 220)
(89, 220)
(80, 220)
(69, 220)
(49, 220)
(53, 220)
(70, 220)
(76, 220)
(75, 220)
(64, 220)
(64, 220)
(134, 220)
(160, 220)
(108, 220)
(158, 220)
(158, 220)
(159, 220)
(160, 220)
(158, 220)
(160, 220)
(158, 220)
(164, 220)
(164, 220)
(175, 220)
(185, 220)
(185, 220)
(188, 220)
(193, 220)
(195, 220)
(198, 220)
(201, 220)
(201, 220)
(197, 220)
(184, 220)
(184, 220)
(185, 220)
(188, 220)
(188, 220)
(191, 220)
(191, 220)
(193, 220)
(194, 220)
(195, 220)
(196, 220)
(196, 220)
(197, 220)
